# Chatbot Hospitality Industry
<hr>

# Imports
<hr>

In [15]:
# Import

import os
import re
import json

import numpy as np
import random
import pickle

#import transformers
import speech_recognition as sr
from gtts import gTTS

from tensorflow import keras
from sklearn.preprocessing import LabelEncoder


### Primers
<hr>

In [16]:
# Key Variables
# This is the main Variable that will have all the towel types and counts

# Variable for the towel data 
towel_req = [[0,0],         #      , room number
             [0,0],         #  flag, count bath towel 
             [0,0],         #  flag, count face towel
             [0,0]]         #  flag, count washcloth


In [17]:
# Key Phrases that will start a menu selection 
## - to be stored on seporate libary 

key_towel = '.*\\bi need some towels\\b.*|.*\\bi need some towel\\b.*' #"I need some towels"

## Speech to Text and Text to Speech Functions
<hr>

In [18]:
# gTTS - Convert Text to speech

def text_to_speech(text):
    print("Virtual Assistant: ", text)
    speaker = gTTS(text=text, lang="en", slow=False)
    speaker.save("res.mp3")
    os.system("afplay res.mp3") 
    os.remove("res.mp3")
    

In [19]:
# Speech Recognition, using speech_recognition

def speech_to_text():
    text = ""
    recognizer = sr.Recognizer()
    with sr.Microphone() as mic:
        print("--- listening now --- ")
        audio = recognizer.listen(mic)
    try:
        text = recognizer.recognize_google(audio)
        print("Human:",text)
    except:
        print("Human: Error no sound recondized")
        #text_to_speech('i am sorry i did not get that.')
    return text.lower()


<br>

#### Towel Loop 


In [20]:
"""
    Loop for towels if the client request towels

"""



def need_towels():
    yes = '.*\\byes\\b.*|.*\\byep\\b.*|.*\\bcorrect\\b.*'
    no =  '.*\\bno\\b.*|.*\\bnegative\\b.*|.*\\bnope\\b.*'
    
    #Var & flags
    #keyRoomNumber = False
    flag = True
    exit = False
    #additional_flag = False
    
    towel_req = [[0,0],         #      , room number
             [0,0],         #  flag, count bath towel 
             [0,0],         #  flag, count face towel
             [0,0]]         #  flag, count washcloth
    
    
    # loop for towel 
    while exit == False:
        
       ### Loop to select more then 1 type of towel
        while (flag == True):
            
           # Type of Towel 
            towel_req = TypeofTowel(towel_req)

           # Would you like additonal towels ? 
            flag = yesNo('would you like any additional type of towel')
                    
    ### Confirm towel amounts          
        exit = towel_confirmation(towel_req)
        
    ### Input Room Number 
    towel_req[0][0] = roomNumber()
    
    
    ### ### ###
    
    return towel_req
        
        

<br>

#### Yes or No-INPUT


In [21]:
"""
    Yes / No input 

"""

def yesNo(vaReply):
    
    yes = '.*\\byes\\b.*|.*\\byep\\b.*|.*\\bcorrect\\b.*'
    no =  '.*\\bno\\b.*|.*\\bnegative\\b.*|.*\\bnope\\b.*'
    
    # Var
    exitLoop = False
        
    while True:
        
        # output 
        text_to_speech(vaReply)
        # input 
        human_input = speech_to_text()
        
       # IF 
    
        if re.search(no,human_input):
            # no
            return False
        elif re.search(yes,human_input):
            # flag to exit loop
            return True
        else:
            text_to_speech("I am sorry i did not understand that.")
    
    return exitLoop

<br>

### Number of towels-INPUT 


In [22]:
"""
    Input Number of Towels
    
""" 


def numberofTowels(type_of_towel):
    #----yes = '.*\\byes\\b.*|.*\\byep\\b.*|.*\\bcorrect\\b.*'
    clean_input = 0     

    vaReply = "how many " + type_of_towel + " towels would you like?"
    text_to_speech(vaReply)
    
    while (True):
    
        # input 
        human_input = speech_to_text()
        # only the numbers
        try:
            clean_input = int(re.sub('\D', '', human_input) )
        except:
            vaReply = "I am sorry i did not get that.  How many " + type_of_towel + " towels would you like?"
            text_to_speech(vaReply)
            print('except')
             
        if (clean_input <= 20) & (clean_input > 0): 
            return clean_input
            
        elif clean_input > 20:
            vaReply = 'i am sorry the max towels we can send you is 20.  We will send you 20.'    
            text_to_speech(vaReply)
            return 20
        else:
            text_to_speech('is this another converstation')
            print('is this another converstation')
    
        

<br>

### Type of Towel - INPUT

In [23]:
"""
    Type of towels selection

"""


def TypeofTowel(towel_req):
    
    # Var
    
    exit = False
    
    
    while (exit == False):
    
        # output 
        vaReply = "what type of towels would you like: bath towels, hand towels, or washcloths"
        text_to_speech(vaReply)
        # Input 
        human_input = speech_to_text()
        

       ### if statment
        if re.search('.*\\bbath\\b.*',human_input):
            towel_req[1][0] = 1 
            exit = True
            # number of towels
            towel_req[1][1] = numberofTowels('bath') 
            
        elif re.search('.*\\bhand\\b.*|.*\\bhands\\b.*',human_input):
            towel_req[2][0] = 1 
            exit = True
            # number of towels
            towel_req[2][1] = numberofTowels('hand') 
            
        elif re.search('.*\\bwashcloth\\b.*',human_input):
            towel_req[3][0] = 1 
            exit = True
            # number of towels
            towel_req[3][1] = numberofTowels('washcloth') 
        
        else:
            print('ELSE')
            #while 
                #smart-chat responce
                # maybe if smart chatbot responce cosi is > ___ threshhold 
            vaReply = "I am sorry i did not understand that"
            text_to_speech(vaReply)
        
    return towel_req      
        

<br>

### Confirm Towels type & amount


In [24]:
"""
   Confirms Towels Type & amount

"""


def towel_confirmation(towel_req):

    
    # VAR
    yes = '.*\\byes\\b.*|.*\\byep\\b.*|.*\\bcorrect\\b.*'
    no =  '.*\\bno\\b.*|.*\\bnegative\\b.*|.*\\bnope\\b.*|.*\\bnot\\b.*'
    
    
    towel_dict = {1:'bath', 2:'hand', 3:'washcloths'}
    words = ['','','']
    conj_words = ['','']
    towels_word = ' towels '
    c = 0
    
    
  ### runs through towel array to Create vaReply sentence
    for i in range(1,4):
        
    # checks agent towel type flag in array 
        if towel_req[i][0] == 1:
            # checks to see if word towels should be used (not using word towels after washcloths)
            if i == 3:
                towels_word  = ''
            words[(i-1)] = str(towel_req[i][1]) + ' ' + towel_dict[i] + towels_word
            c += 1
            
    # chooses conjections for sentence         
    if c == 2:
        conj_words[0] = 'and '
    elif c == 3:
        conj_words[0] = ','
        conj_words[1] = 'and '
    
    # appends sentence
    vaReply = 'to confirm, you would like ' + words[0] + conj_words[0] + words[1] + conj_words[1] + words[2] + ', is this correct?'    
    
    while True:
        
        # output
        text_to_speech(vaReply)
        
        # input 
        human_input = speech_to_text()
        

        if re.search(no,human_input):
            vaReply = "no problem let me change that"
            text_to_speech(vaReply)
            # causes the loop to run again 
            return False
            
        elif re.search(yes,human_input):
            vaReply = "perfect"
            text_to_speech(vaReply)
            # exits the loop 
            return True 
        else:
            text_to_speech('I am sorry i did not get that') 

<br>

### Room Number 

In [25]:
"""
    Enters input for Room number

"""
def roomNumber():
   # VAR 
    flag = False
    keyRoomNumber = False
    clean_input = 0   

    vaReply = "What room number should I send this up too?"
    #text_to_speech(vaReply)
    
    while (keyRoomNumber == False):
        text_to_speech(vaReply)
       
        # input 
        human_input = speech_to_text()
        
        try:
            clean_input = int(re.sub('\D', '', human_input) )
            print('try - clean input: ',clean_input)
            
        except:
            vaReply = "I am sorry i did not get that, what is your room number?"
            text_to_speech(vaReply)
            print('except')
        
        if (clean_input <= 5000) & (clean_input > 0): 
            flag = True
            room_number = clean_input 
        elif (clean_input > 5000) & (clean_input < 0):
            text_to_speach('I am sorry that room number does not exist')
            
            
       # FLAG to confirm room number 
        if flag == True:
            keyRoomNumber , flag = room_confirmation(room_number)
    
    return room_number
            
            

<br>

### Confirm Room Number

In [26]:
"""
   Confirms Room numbers

"""


def room_confirmation(room_number):
   # VAR
    yes = '.*\\byes\\b.*|.*\\byep\\b.*|.*\\bcorrect\\b.*'
    no =  '.*\\bno\\b.*|.*\\bnegative\\b.*|.*\\bnope\\b.*|.*\\bnot\\b.*'
    
    vaReply = "you said room " + str(room_number) + ' is that correct?'
    text_to_speech(vaReply)
    
    while True:
        
        # input 
        human_input = speech_to_text()
        print('human input: ',bool(re.search(yes,human_input)))
        if re.search(no,human_input):
            vaReply = "no problem let me change that"
            text_to_speech(vaReply)
            return False, False
            
        elif re.search(yes,human_input):
            vaReply = "perfect"
            text_to_speech(vaReply)
            return True,True 
        else:
            vaReply = "I am sorry i did not understand that, you said room " + str(room_number) + ' is that correct?'
            text_to_speech(vaReply) 
        print('confirm loop')

# Main
<hr>

In [27]:
### Main program

if __name__ == "__main__":
    
    
    # flags
    mainloopflag = True
    new_topic = True

    # load data 
    with open("../data/intents.json") as file:
        data = json.load(file)

    # load trained model
    model = keras.models.load_model('../assets/chat_model')

    # load tokenizer object
    with open('../assets/tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
    
    # load label encoder object
    with open('../assets/label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20   

    #Welcome
    text_to_speech('Hello this is the Tokyo Grand Hyatt, how can we help you')

    # main loop 
    while (mainloopflag == True):
        # Welcome 
            

            
        res = speech_to_text()
        
        
    ### Menu System 

        """
        NoteToSelf::Clean-up below code to be more fluid (if else if). 
            Need to blend the rulebased chat and the connersational chat
        """
    
    # NoteToSelf::Create into funciton 

        # pass input to pickle models
        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([res]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        #check for resonse
        for i in data['intents']:
            if i['tag'] == tag:
                response_i = i
                response_true = True
                break
    

        
        # I need some Towels
        if re.search(key_towel, res):
            towel_req = need_towels()
            text_to_speech('Excelent, we will send them straight up!')
            print(towel_req)
        
        # converstaional 
        elif response_true == True:
            text_to_speech( np.random.choice(response_i['responses']) )

        else:
            text_to_speech('I am sorry I did not get that')
            
     

        
        # Exit call 
        to_exit = yesNo('can i help you with anything else')
        if to_exit == False:
            text_to_speech('Thank you and have a beautiful day')
            break
        else:
            text_to_speech('How can i help you?')
                
            
        
            
            


Virtual Assistant:  Hello this is the Tokyo Grand Hyatt, how can we help you


In [ ]:

for number in range(10):
    if number == 5:
        break    # break here

    print('Number is ' + str(number))

print('Out of loop')


Number is 0
Number is 1
Number is 2
Number is 3
Number is 4
Out of loop


In [1]:
"hello Mike".find("Mike") 

6